# Produce a XR-clean kDST

<i>
This notebook takes a kDST and filters the data to produce
a XR-clean kDST in order to facilitate the Kr analysis.
</i>

<i>
In presence of external gamma sources, the XR production
is very significative and constitutes a source of
background for Kr data. This NB filters the data to
produce a rather pure dataset on which to perform
the regular analysis.
</i>

### Notebook configuration

In [ ]:
run_number            = XXX_RUN_NUMBER_XXX
run_number_correction = 6165
dst_selection        = "ezselection"

input_dst_filename  = f"$IC_DATA/{run_number}/kdst/kdst_{run_number}.h5"
correction_filename = f"$IC_DATA/XYmaps/corrections_run{run_number_correction}.h5"
output_dst_filename = f"$IC_DATA/{run_number}/dst/kdst_{run_number}_{dst_selection}.h5"

apply_selection = True

n_sigma = 3.5
Zrange  =   0,  550
Zfit    =  50,  500
Erange  = 2e3, 20e3
Znbins  =  50
Enbins  = 100

# Plotting style
#default_cmap = "viridis"
figure_size  = 14, 10
font_size    = 12
with_titles  = True

### Imports

In [ ]:
import os
import time

import tables            as tb
import numpy             as np
import matplotlib.pyplot as plt

#import invisible_cities.core.fit_functions as fitf
import invisible_cities.reco.dst_functions as dstf

from invisible_cities.core .core_functions import in_range
from invisible_cities.icaro.hst_functions  import shift_to_bin_centers

import core.corrections
import core.kr_plt_functions as pkr
import core.kr_ana_functions as akr

%matplotlib inline
%load_ext autoreload
%autoreload 2

### Initialization

In [ ]:
print("This notebook has been run on ", time.asctime())

In [ ]:
input_dst_filename  = os.path.expandvars( input_dst_filename)
correction_filename = os.path.expandvars(correction_filename)
output_dst_filename = os.path.expandvars(output_dst_filename)

plt.rcParams["figure.figsize"] = figure_size
plt.rcParams[  "font.size"   ] =   font_size

### Read data

In [ ]:
dst           = dstf.load_dst(input_dst_filename, "DST", "Events")
dst           = dst[dst.nS1 == 1]

number_of_S2s_full, number_of_evts_full = akr.kdst_unique_events(dst)

print(f"Total number of S2s   : {number_of_S2s_full} ")
print(f"Total number of events: {number_of_evts_full}")
dst.head()

In [ ]:
if "index" in dst: del dst["index"]

### Data filtering

In [ ]:
X   = dst.X  .values
Y   = dst.Y  .values
Z   = dst.Z  .values
T   = dst.time.values - dst.time.values.min()
S2e = dst.S2e
E = corrections.Ecorrection(correction_filename, S2e, X, Y, Z)
# if apply also Z-correction - uncomment
# E = corrections.Ecorrection(correction_filename, S2e, X, Y, Z);

In [ ]:
sel_inband = akr.selection_in_band(E, Z, Erange, Zrange, Zfit, plot=True);
nsel   = np.sum(sel_inband)
effsel = 100.*nsel/(1.*len(sel_inband)) 
print(f"Total number of selected candidates : {nsel} ({effsel:.1f} %)" )

### Produce filtered kDST

In [ ]:
# subdst = dst[in_range(E, lowE_cut(Z), highE_cut(Z))]
subdst = dst[sel_inband]

number_of_S2s_filtered, number_of_evts_filtered = akr.kdst_unique_events(subdst)

number_of_S2s_ratio  = number_of_S2s_filtered  / number_of_S2s_full  * 100
number_of_evts_ratio = number_of_evts_filtered / number_of_evts_full * 100

print(f"Total number of S2s   : {number_of_S2s_filtered} ({number_of_S2s_ratio:.1f} %)" )
print(f"Total number of events: {number_of_evts_filtered} ({number_of_evts_ratio:.1f} %)")

### Comparison between original and filtered data

In [ ]:
pkr.dst_compare_vars(dst, subdst)

In [ ]:
akr.kdst_write(subdst, output_dst_filename);